In [1]:
import numpy as np 
import pandas as pd  
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.datasets import load_files
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

train_path = './train'
validate_path = './val'
test_path = './test'


Using TensorFlow backend.


In [2]:

dim = 299
epochs = 50
learning_rate = 0.0001
batch_size = 16

train_num = 3790
validation_num = 480


In [3]:

weights = os.path.join('', 'weights.h5')

datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1)


In [4]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

validate_generator = datagen.flow_from_directory(
    validate_path,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode='categorical')

Found 3790 images belonging to 12 classes.
Found 480 images belonging to 12 classes.


In [5]:

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
             ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=1),
             ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001,
                               cooldown=0, min_lr=0)]

In [6]:

base_model = Xception(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg')
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


In [7]:

model.fit_generator(train_generator,
                    steps_per_epoch=train_num / batch_size,
                    validation_data=validate_generator,
                    validation_steps=validation_num/ batch_size,
                    callbacks=callbacks,
                    epochs=epochs,
                    verbose=1)

Epoch 1/50
237/236 [==============================] - 353s - loss: 0.9008 - acc: 0.7320 - val_loss: 0.3476 - val_acc: 0.8667
Epoch 2/50
237/236 [==============================] - 340s - loss: 0.2394 - acc: 0.9217 - val_loss: 0.1853 - val_acc: 0.9333
Epoch 3/50
237/236 [==============================] - 337s - loss: 0.1488 - acc: 0.9517 - val_loss: 0.2331 - val_acc: 0.9187
Epoch 4/50
237/236 [==============================] - 339s - loss: 0.1237 - acc: 0.9618 - val_loss: 0.1619 - val_acc: 0.9500
Epoch 5/50
237/236 [==============================] - 337s - loss: 0.0965 - acc: 0.9704 - val_loss: 0.1710 - val_acc: 0.9500
Epoch 6/50
237/236 [==============================] - 340s - loss: 0.0827 - acc: 0.9751 - val_loss: 0.1446 - val_acc: 0.9521
Epoch 7/50
237/236 [==============================] - 338s - loss: 0.0706 - acc: 0.9810 - val_loss: 0.1804 - val_acc: 0.9458
Epoch 8/50
237/236 [==============================] - 337s - loss: 0.0570 - acc: 0.9834 - val_loss: 0.1852 - val_acc: 0.9437


In [51]:


data_test = load_files(test_path)
test_label = np.array(data_test['target'])
print(np.array(data_test['target']))


weights = os.path.join('', 'weights.h5')
test_datagen = ImageDataGenerator()       
generator_test = test_datagen.flow_from_directory(
                                            test_path,
                                            target_size=(dim, dim),
                                            batch_size=batch_size,
                                            class_mode=None, 
                                            shuffle=False) 

    

[ 0  3  3  6  4  3  9 10  2  0  8  2 11  8  6  5  1  2  7  8 11  2  6  0  9
  7  5 11 10  8 10  0 10  8  9  4  7  6  7  9  0  1 11 10  3  0  8  6  6  4
  4  5  7  3  5  2  5  8  8  7 11 11  7 10  4  5  7  1 10  8  3  1 11  5  6
  7 11  1  4  1  4  1  9  0  2 10  5  7  0  9  0 10 10  2 11  4  9  6  5  5
  5  6  1  7  9 11  9 11  1  6  1  3  1  4  4 11  4  3  5 11 11 10  9  3  3
  1  7  1  1  0  6  0  7  0  5  3 10  3  0  1  0  7  0  7  8  1  4  8  2  8
  8  3 11 10 11  8 11  7  6  1  5  8 10  2  7  1 11 10 10  6  2  9  2  7  7
  6  2  0  3  3  3  7 11  5  5  9  1  7  5  3  2  9  0  9  0  3  8  6  0  2
  5  2  6  7  4  5  2  4 10  3  6  2  9  7  7  3  4  8  6  3 10  5  9  9  8
  3  0 11  4  0  1  2  2  1  4  4  4  7  8 11 10  8  2  9  5  1 10  6 10  1
  3 11  0  9  0  4  8  8 10  5  0 11  7  3  9  9  3  5  2  8 10  8  5  9  4
  5  5  5  6  1  0  5  5  3  2  4 10  9  1  6  8  1 10 11  0  7  9 11  5  4
  2 11 11  2  3  3  0  8  2  9  3  5  6  8  9  6  9  0  7  6  5 10  0 11  3
  7  7  1  4

In [52]:

filenames = generator_test.filenames
nb_samples = len(filenames)
print(nb_samples)
prediction_test = model.predict_generator(generator_test, verbose=1,steps =30)

480
30/30 [==============================] - 13s    


In [55]:
print("done")



done


In [56]:

prediction_list = []
for i in range(len(prediction_test)):
    result = np.argmax(prediction_test[i])
    prediction_list.append(result)

count = 0

for i in range(len(prediction_list)):
    if prediction_list[i] == generator_test.classes[i]:
        
        count += 1
print(count/480)

480
480
0.9645833333333333
